# SQL: Chinook Database Problems 

This problem set is from Dataquest.io and utilizes the popular Chinook sample database. 

In [1]:
from IPython.display import Image
from IPython.core.display import HTML 
print("ER Diagram of the Chinook Database:")
Image(url= "https://blog.xojo.com/wp-content/uploads/2016/04/ChinookDatabaseSchema1.1.png")


ER Diagram of the Chinook Database:


In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

## Task 1: Determining the Most Popular Music Genres in the USA

In [3]:
%%sql
with t1 AS
  (SELECT distinct g.name AS genre_name,
                   count(t.track_id) AS num_sales
   FROM invoice i
   inner join invoice_line il
     ON i.invoice_id = il.invoice_id
   inner join track t
     ON il.track_id = t.track_id
   inner join genre g
     ON t.genre_id = g.genre_id
   WHERE billing_country = 'USA'
   GROUP BY genre_name),
     total_sales AS
  (SELECT sum(num_sales) AS total
   FROM t1)
SELECT t1.genre_name,
       t1.num_sales,
       t1.num_sales*100/total_sales.total AS percent_sales
FROM t1,
     total_sales
GROUP BY genre_name,
         num_sales
ORDER BY num_sales DESC;

 * sqlite:///chinook.db
Done.


genre_name,num_sales,percent_sales
Rock,561,53
Alternative & Punk,130,12
Metal,124,11
R&B/Soul,53,5
Blues,36,3
Alternative,35,3
Latin,22,2
Pop,22,2
Hip Hop/Rap,20,1
Jazz,14,1


Chinook should focus on acquiring music in the Rock, Alt/Punk, and Metal genres; other genres do not sell nearly as well, so only top albums from these genres should be acquired. 



## Task 2: Determining Top-Performing Salesperson

In [4]:
%%sql

SELECT 
    e.employee_id,
    e.first_name || " " || e.last_name AS name,
    round(sum(i.total),2) AS sales_amount,
    count(c.customer_id) AS num_clients,
    round(sum(i.total)/count(c.customer_id),2)  AS sale_per_client
FROM employee e
   inner join customer c
     ON e.employee_id = c.support_rep_id
    inner join invoice i
     ON c.customer_id=i.customer_id
    GROUP BY employee_id,
            name

 * sqlite:///chinook.db
Done.


employee_id,name,sales_amount,num_clients,sale_per_client
3,Jane Peacock,1731.51,212,8.17
4,Margaret Park,1584.0,214,7.4
5,Steve Johnson,1393.92,188,7.41


Looking at raw sales numbers, it appears at first glance that Steve Johnson is the worst performing salesman. However, if you look at the number of clients that each salesperson has, it becomes clear that he has fewer clients than either of the women. His performance is on par with Margaret when you look at the amount each client is spending. So he is not the worst salesman - Jane is just the best.

## Task 3: Comparing Sales Data by Country
For the purposes of this analysis, all countries with only one customer will be grouped into the category "other". 

In [5]:
%%sql
with t1 AS
  (SELECT billing_country AS country,
          count(distinct customer_id) AS num_customers,
          round(sum(total),2) AS total,
          round(avg(total),2) AS avg_order_spend,
          count(invoice_id) AS num_orders
   FROM invoice
   GROUP BY country
   ORDER BY num_customers DESC),
     t2 AS
  (SELECT CASE
              WHEN num_customers>1 then country
              ELSE "other"
          END country,
          num_customers,
          total,
          avg_order_spend,
          num_orders
   FROM t1
   GROUP BY country),
     t3 AS
  (SELECT country,
          sum(num_customers) AS num_customers,
          sum(total) AS total_sales,
          avg_order_spend,
          sum(num_orders) AS num_orders
   FROM t2
   GROUP BY t2.country)
SELECT country,
       num_customers,
       num_orders,
       round(num_orders/num_customers,2) AS orders_per_customer,
       total_sales,
       round(total_sales/num_customers,2) AS avg_cust_spend,
       avg_order_spend
FROM t3
ORDER BY num_customers DESC;



 * sqlite:///chinook.db
Done.


country,num_customers,num_orders,orders_per_customer,total_sales,avg_cust_spend,avg_order_spend
other,15,147,9.0,1094.94,73.0,7.92
USA,13,131,10.0,1040.49,80.04,7.94
Canada,8,76,9.0,535.59,66.95,7.05
Brazil,5,61,12.0,427.68,85.54,7.01
France,5,50,10.0,389.07,77.81,7.78
Germany,4,41,10.0,334.62,83.66,8.16
United Kingdom,3,28,9.0,245.52,81.84,8.77
Czech Republic,2,30,15.0,273.24,136.62,9.11
India,2,21,10.0,183.15,91.58,8.72
Portugal,2,29,14.0,185.13,92.57,6.38


## Task 4: Album Purchase vs. Track Purchase

We are trying to determine if it is worth it for Chinook to continue to acquire whole albums from the distributors. What is the return on investment? What percentage of purchases are whole albums, as opposed to individual tracks?

**Note: Since a small number of albums contain only a few tracks (such as singles or EPs), we will be selecting only for full-length albums with five or more songs.

In [6]:
%%sql
with album_length AS
  (SELECT album_id,
          count(track_id) AS length
   FROM track
   GROUP BY album_id),
           album_info AS
  (SELECT count(il.track_id) AS num_tracks_from_album,
          ab.length,
          t.album_id,
          il.invoice_id
   FROM track t
   inner join invoice_line il
     ON t.track_id = il.track_id
   inner join album_length ab
     ON ab.album_id = t.album_id
   GROUP BY t.album_id,
            il.invoice_id
   ORDER BY invoice_id),
           album_bool AS
  (SELECT invoice_id,
          CASE
              WHEN num_tracks_from_album = length
                   AND length>=5 then 1
              ELSE 0
          END album_t_f
   FROM album_info),
           album_bool_2 AS
  (SELECT invoice_id,
          sum(album_t_f) AS bool
   FROM album_bool
   GROUP BY invoice_id),
           total_invoices AS
  (SELECT count(distinct invoice_id) AS total
   FROM album_bool)
SELECT CASE
           WHEN bool > 0 then 'Album'
           ELSE 'Tracks Only'
       END purchase_type,
       count(invoice_id) AS num_invoices,
       count(invoice_id)*100/total AS percent
FROM album_bool_2,
     total_invoices
GROUP BY purchase_type

 * sqlite:///chinook.db
Done.


purchase_type,num_invoices,percent
Album,110,17
Tracks Only,504,82


When looking at the number of invoices that contain whole albums vs. only tracks, it is obvious that most purchases do not include whole albums. Still, 17% of sales *are* whole albums. To decide if it is a good investment to continue buying whole albums, I think it is worth it to look further into the sales numbers for both types of purchases. I'm going to modify my code to include purchase data in the query.


In [7]:
%%sql with album_length AS
  (SELECT album_id,
          count(track_id) AS length
   FROM track
   GROUP BY album_id),
           album_info AS
  (SELECT count(il.track_id) AS num_tracks_from_album,
          ab.length,
          t.album_id,
          il.invoice_id,
          i.total AS invoice_total
   FROM track t
   inner join invoice_line il
     ON t.track_id = il.track_id
   inner join album_length ab
     ON ab.album_id = t.album_id
   inner join invoice i
     ON il.invoice_id = i.invoice_id
   GROUP BY t.album_id,
            il.invoice_id,
            i.total
   ORDER BY il.invoice_id),
           album_bool AS
  (SELECT invoice_id,
          invoice_total,
          CASE
              WHEN num_tracks_from_album = length
                   AND length>=5 then 1
              ELSE 0
          END album_t_f
   FROM album_info),
           album_bool_2 AS
  (SELECT invoice_id,
          sum(album_t_f) AS bool,
          invoice_total
   FROM album_bool
   GROUP BY invoice_id,
            invoice_total),
           total_invoices AS
  (SELECT count(distinct invoice_id) AS total_num_invoices
   FROM album_bool_2),
           total_sale_amt AS
  (SELECT sum(invoice_total) AS total_invoice_sales
   FROM album_bool_2)
SELECT CASE
           WHEN bool > 0 then 'Album'
           ELSE 'Tracks Only'
       END purchase_type,
       count(invoice_id) AS num_invoices,
       count(invoice_id)*100/total_num_invoices AS percent_invoices,
       round(avg(invoice_total),2) AS avg_invoice_amt,
       round(sum(invoice_total),2) AS total_sales,
       round(sum(invoice_total)*100/total_invoice_sales, 2) AS percent_sales
FROM album_bool_2,
     total_invoices,
     total_sale_amt
GROUP BY purchase_type

 * sqlite:///chinook.db
Done.


purchase_type,num_invoices,percent_invoices,avg_invoice_amt,total_sales,percent_sales
Album,110,17,13.23,1455.3,30.9
Tracks Only,504,82,6.46,3254.13,69.1


Although album sales are only 17% of invoices, the average invoice amount for album purchases is double the amount of purchases without albums. Album purchases are almost 1/3 of total sales, despite being only 17% of the invoice count. Because of that disparity, Chinook may lose high-spending clients if they choose to stop buying whole albums from the distributor. Unless the company really needs to cut costs, I think it is a better idea to continue purchasing whole albums.
